# Data Integration with BDI

First, import the class `APIManager`.

In [1]:
from bdi import APIManager

Add the path to the pre-trained model for mapping recommendations.

In [2]:
import os
os.environ['BDI_MODEL_PATH'] = '/Users/rlopez/Downloads/model_20_1.pt' #YOUR PATH HERE

## Dataset Loading

In this example, we are mapping the Dou dataset to the GDC format.

In [3]:
manager = APIManager()

In [4]:
dataset_path =  './datasets/dou.csv'
dataset = manager.load_dataset(dataset_path)
dataset

,Country,Histologic_Grade_FIGO,Histologic_type,Path_Stage_Primary_Tumor-pT,Path_Stage_Reg_Lymph_Nodes-pN,Clin_Stage_Dist_Mets-cM,Path_Stage_Dist_Mets-pM,tumor_Stage-Pathological,FIGO_stage,BMI,Age,Race,Ethnicity,Gender,Tumor_Site,Tumor_Focality,Tumor_Size_cm
0,United States,FIGO grade 1,Endometrioid,pT1a (FIGO IA),pN0,cM0,Staging Incomplete,Stage I,IA,38.88,64.0,White,Not-Hispanic or Latino,Female,Anterior endometrium,Unifocal,2.9
1,United States,FIGO grade 1,Endometrioid,pT1a (FIGO IA),pNX,cM0,Staging Incomplete,Stage IV,IA,39.76,58.0,White,Not-Hispanic or Latino,Female,Posterior endometrium,Unifocal,3.5
2,United States,FIGO grade 2,Endometrioid,pT1a (FIGO IA),pN0,cM0,Staging Incomplete,Stage I,IA,51.19,50.0,White,Not-Hispanic or Latino,Female,"Other, specify",Unifocal,4.5
3,NaN,NaN,Carcinosarcoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,FIGO grade 2,Endometrioid,pT1a (FIGO IA),pNX,cM0,No pathologic evidence of distant metastasis,Stage I,IA,32.69,75.0,White,Not-Hispanic or Latino,Female,"Other, specify",Unifocal,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Reducing the GDC Scope

Reduce the scope of GDC selecting top k candidates for each column.

In [5]:
manager.reduce_scope()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 17.98it/s]


Table features extracted from 17 columns


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 734/734 [01:25<00:00,  8.58it/s]

Table features extracted from 734 columns

Country:


,Candidate,Similarity,Description,Values (sample)
0,country_of_birth,0.5726,The name of the country in which the patient i...,"Afghanistan, Albania, Algeria"
1,country_of_residence_at_enrollment,0.5151,The text term used to describe the patient's c...,"Afghanistan, Albania, Algeria"
2,variant_origin,0.3803,The text term used to describe the biological ...,"Germline, Somatic, Unknown"
3,zone_of_origin_prostate,0.3563,The location or position of the tumor by zone ...,"Central zone, Overlapping/multiple zones, Peri..."
4,tumor_confined_to_organ_of_origin,0.3322,The yes/no/unknown indicator used to describe ...,"Yes, No, Unknown"
5,race,0.2936,An arbitrary classification of a taxonomic gro...,"american indian or alaska native, asian, black..."
6,vascular_invasion_present,0.291,The yes/no indicator to ask if large vessel or...,"Yes, No, Unknown"
7,lymphatic_invasion_present,0.287,A yes/no indicator to ask if small or thin-wal...,"Yes, No, Unknown"
8,ethnicity,0.2618,An individual's self-described social and cult...,"hispanic or latino, not hispanic or latino, Un..."
9,perineural_invasion_present,0.2578,a yes/no indicator to ask if perineural invasi...,"Yes, No, Unknown"


...


Tumor_Focality:


,Candidate,Similarity,Description,Values (sample)
0,tumor_focality,0.808,The text term used to describe whether the pat...,"Multifocal, Unifocal, Unknown"
1,tumor_shape,0.5626,Text term to represent the description of the ...,"Diffuse, Dome, Mushroom"
2,tumor_depth_descriptor,0.5311,Text term for the degree to which a tumor has ...,"Deep, Superficial, Not Reported"
3,enneking_msts_tumor_site,0.4829,Text term and code that represents the tumor s...,"Extracompartmental (T2), Intracompartmental (T..."
4,biospecimen_type,0.4809,The text term used to describe the biological ...,"Blood, Bone Marrow, Buccal Mucosa"
5,tissue_type,0.4783,Text term that represents a description of the...,"Abnormal, Normal, Peritumoral"
6,wilms_tumor_histologic_subtype,0.4597,The text term used to describe the classificat...,"Favorable, Unfavorable, Unknown"
7,residual_tumor_measurement,0.4544,A measurement of the tumor cells that remain i...,"1-10 mm, 11-20 mm, >20 mm"
8,non_nodal_tumor_deposits,0.451,The yes/no/unknown indicator used to describe ...,"Yes, No, Unknown"
9,tumor_infiltrating_macrophages,0.4465,Non-neoplastic macrophages that infiltrate a t...,"Few, Many, Moderate"



Tumor_Size_cm:


,Candidate,Similarity,Description,Values (sample)
0,shortest_dimension,0.7575,Numeric value that represents the shortest dim...,
1,size_extraocular_nodule,0.7416,The size of the nodule that is outside the eye.,
2,tumor_width_measurement,0.7034,The numerical measurement of tumor width.,
3,tumor_depth_measurement,0.6989,The numerical measurement of tumor depth.,
4,tumor_thickness,0.6506,A measurement of the thickness of a sectioned ...,
5,analyte_quantity,0.6418,The quantity in micrograms (ug) of the analyte...,
6,average_insert_size,0.6362,Average insert size collected from samtools.,
7,tumor_largest_dimension_diameter,0.5736,Numeric value used to describe the maximum dia...,
8,mitotic_total_area,0.5635,The total area reviewed when calculating the m...,
9,rin,0.5578,A numerical assessment of the integrity of RNA...,


[{'Candidate column': 'Country',
  'Top k columns': [('country_of_birth', '0.5726'),
   ('country_of_residence_at_enrollment', '0.5151'),
   ('variant_origin', '0.3803'),
   ('zone_of_origin_prostate', '0.3563'),
   ('tumor_confined_to_organ_of_origin', '0.3322'),
   ('race', '0.2936'),
   ('vascular_invasion_present', '0.291'),
   ('lymphatic_invasion_present', '0.287'),
   ('ethnicity', '0.2618'),
   ('perineural_invasion_present', '0.2578'),
   ('lymph_node_involvement', '0.2574'),
   ('ovarian_surface_involvement', '0.2449'),
   ('is_legacy', '0.2389'),
   ('satellite_nodule_present', '0.2356'),
   ('releasable', '0.2233'),
   ('consent_type', '0.2182'),
   ('oct_embedded', '0.2094'),
   ('id', '0.201'),
   ('gender', '0.1994'),
   ('necrosis_present', '0.1991')]},
 {'Candidate column': 'Histologic_Grade_FIGO',
  'Top k columns': [('histologic_progression_type', '0.6556'),
   ('who_nte_grade', '0.5967'),
   ('tumor_grade', '0.5817'),
   ('tumor_grade_category', '0.5759'),
   ('inpc

## Column Mapping

Perform column mapping.

In [8]:
column_mappings = manager.map_columns()

,Original Column,Target Column
0,Country,country_of_birth
1,Histologic_Grade_FIGO,histologic_progression_type
2,Histologic_type,history_of_tumor_type
3,Path_Stage_Primary_Tumor-pT,ajcc_pathologic_stage
4,Path_Stage_Reg_Lymph_Nodes-pN,ajcc_pathologic_stage
5,Clin_Stage_Dist_Mets-cM,uicc_clinical_stage
6,Path_Stage_Dist_Mets-pM,ajcc_pathologic_stage
7,tumor_Stage-Pathological,ajcc_pathologic_stage
8,FIGO_stage,figo_stage
9,BMI,bmi


## Value Mapping

Perform value mapping.

In [9]:
value_mappings = manager.map_values()


Column tumor_Stage-Pathological:


,Current Value,Target Value,Similarity
0,Stage I,Stage I,1.0
1,Stage IV,Stage IV,1.0
2,Stage III,Stage III,1.0
3,Stage II,Stage II,1.0
4,nan,-,-



Column Race:


,Current Value,Target Value,Similarity
0,White,white,1.0
1,Asian,asian,1.0
2,Not Reported,not reported,1.0
3,Black or African American,black or african american,1.0
4,nan,-,-



Column Tumor_Focality:


...

,Current Value,Target Value,Similarity
0,IIIC2,Stage IIIC2,1.0
1,IIIC1,Stage IIIC1,1.0
2,IVB,-,-
3,nan,-,-
4,II,-,-
5,IIIA,-,-
6,IIIB,-,-
7,IA,-,-
8,IB,-,-



Column Histologic_type:


,Current Value,Target Value,Similarity
0,Clear cell,Colorectal Cancer,1.0
1,nan,-,-
2,Endometrioid,-,-
3,Serous,-,-
4,Carcinosarcoma,-,-



Column Path_Stage_Primary_Tumor-pT:


,Current Value,Target Value,Similarity
0,pT3a (FIGO IIIA),Stage IIIA,1.0
1,nan,-,-
2,pT1 (FIGO I),-,-
3,pT2 (FIGO II),-,-
4,pT3b (FIGO IIIB),-,-
5,pT1a (FIGO IA),-,-
6,pT1b (FIGO IB),-,-
